In [1]:
import numpy as np
import os
from IPython.display import Markdown, display
from qiskit import QuantumCircuit, QuantumRegister, transpile
from qclib.state_preparation import UCGInitialize
from datetime import datetime
import time
import qc
import qiskit

n_qubits = 10
params = {'protein': 'homo_sapien', 'num_protein': 5, 'num_qubits':10, 'Avg_CX': 0, 'Avg_Runtime(s)': 0}


coupling_map = []
for i in range(n_qubits - 1):
    coupling_map.append([i, i + 1])
    coupling_map.append([i + 1, i])


CX_isa_counts = []
Runtimes_isa = []
CX_vqc_counts = []
Runtimes_vqc = []
CX_ucg_counts = []
Runtimes_ucg = []
for i in range(params['num_protein']):
    name_exp = datetime.now().strftime("%m_%d_%H_%M_%S")
    directory_path = f'Results/Exp:_{name_exp}'
    output_isa_path = f'Results/Exp:_{name_exp}/Outputs_isa'
    output_vqc_path = f'Results/Exp:_{name_exp}/Outputs_vqc'
    os.makedirs(directory_path, exist_ok=True)
    os.makedirs(output_isa_path, exist_ok=True)
    os.makedirs(output_vqc_path, exist_ok=True)
          
    for s in ['isa','vqc']:
        start_time = time.time()
        if f"{s}_cpp" not in os.listdir():
            os.system("./build_cpp")

        
        state_file = f"./States/state{i}.txt"
        output_file = f"Results/Exp:_{name_exp}/Outputs_{s}/gates{i}.txt"
        os.system(f"./{s}_cpp ./{output_file} ./{state_file} {n_qubits} 0.95")

        runtime = time.time() - start_time
        # print(f"Runtime: {runtime}")
        #Now read the gates from the output file
        gate_sequence = []
        with open(output_file) as f:
            while True:
                line = f.readline()
                if len(line) == 0: break
                if len(line) < 3: continue
                [gate_type, a1, a2] = line.split()
                if gate_type == "rx": 
                    gate_sequence.append(qc.Gate.RX(float(a2), int(a1), 5))
                elif gate_type == "ry":
                    gate_sequence.append(qc.Gate.RY(float(a2), int(a1), 5))
                elif gate_type == "rz":
                    gate_sequence.append(qc.Gate.RZ(float(a2), int(a1), 5))
                elif gate_type == "cx" or gate_type == "xc":
                    gate_sequence.append(qc.Gate.CX(int(a1), int(a2), 5))

        CX_count = 0
        for gate in gate_sequence:
            # print(gate.to_string())
            if gate.gate_type == "cx" or gate.gate_type == "xc":
                CX_count += 1

        if s == 'isa':
            CX_isa_counts.append(CX_count)
            Runtimes_isa.append(runtime)
        else:
            CX_vqc_counts.append(CX_count)
            Runtimes_vqc.append(runtime)



   
    state = []
    with open(state_file, 'r') as file:
        for line in file:
            columns = line.split()  
            if len(columns) >= 3:  
    
                amp = float(columns[1])  
                phase = float(columns[2])  
                if phase == 0.0:  
                    state.append(amp)
                else:
                    state.append(-1*amp)
    

    state = state/np.linalg.norm(state)
    ucg_time = time.time()

    # qclib ucg
    circuit_ucg = QuantumCircuit(n_qubits)
    UCGInitialize.initialize(circuit_ucg, state)


    
    circuits = [ ('ucg', circuit_ucg) ]


   
    transpiled = qiskit.transpile(circuit_ucg, basis_gates=['u', 'cx'], coupling_map=coupling_map, optimization_level=3)
    ucg_runtime = time.time() - ucg_time
    Runtimes_ucg.append(ucg_runtime)
    cx = transpiled.count_ops().get('cx', 0)
    CX_ucg_counts.append(cx)

def ave(list):
    return sum(list)/len(list)

table = '| method | qubits | cnots | runtime |\n'
table += '| --- |:---:|:---:|:---:|\n'
table += f'| isa | {n_qubits} | {ave(CX_isa_counts)} | {ave(Runtimes_isa)} |\n'
table += f'| vqc | {n_qubits} | {ave(CX_vqc_counts)} | {ave(Runtimes_vqc)} |\n'
table += f'| ucg | {n_qubits} | {ave(CX_ucg_counts)} | {ave(Runtimes_ucg)} |\n'


display(Markdown(table))

| method | qubits | cnots | runtime |
| --- |:---:|:---:|:---:|
| isa | 10 | 1420.2 | 0.7174319744110107 |
| vqc | 10 | 51.0 | 83.52064890861512 |
| ucg | 10 | 2658.0 | 2.858856439590454 |


In [8]:
import numpy as np
import numpy as np
import os
from IPython.display import Markdown, display
from qiskit import QuantumCircuit, QuantumRegister, transpile
from qclib.state_preparation import UCGInitialize
from datetime import datetime
import time
import qc
import qiskit
n_qubits = 10
CX_ucg_counts = []
Runtimes_ucg = []

params = {'protein': 'homo_sapien', 'num_protein': 5, 'num_qubits':10, 'Avg_CX': 0, 'Avg_Runtime(s)': 0}

coupling_map = []
for i in range(n_qubits - 1):
    coupling_map.append([i, i + 1])
    coupling_map.append([i + 1, i])


params['num_protein'] = 10
for i in range(params['num_protein']):
    state_file = f"./States/state{i}.txt"
    state = []
    with open(state_file, 'r') as file:
        for line in file:
            columns = line.split()  
            if len(columns) >= 3:  

                amp = float(columns[1])  
                phase = float(columns[2])  
                if phase == 0.0:  
                    state.append(amp)
                else:
                    state.append(-1*amp)


        state = state/np.linalg.norm(state)
        ucg_time = time.time()

        # qclib ucg
        circuit_ucg = QuantumCircuit(n_qubits)
        UCGInitialize.initialize(circuit_ucg, state)



        circuits = [ ('ucg', circuit_ucg) ]



        transpiled = qiskit.transpile(circuit_ucg, basis_gates=['u', 'cx'], coupling_map=coupling_map, optimization_level=3)
        ucg_runtime = time.time() - ucg_time
        Runtimes_ucg.append(ucg_runtime)
        cx = transpiled.count_ops().get('cx', 0)
        CX_ucg_counts.append(cx)
print(CX_ucg_counts)

[2662, 2666, 2664, 2664, 2664, 2662, 2666, 2668, 2662, 2645]
